<a href="https://colab.research.google.com/github/makropolecenia/Book/blob/master/Scraper_Flightera_mail_stan_na_20_03_2024_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Scraper Flightera.net oparty o bibliotekę beautifull Soup - stan na 20.03.2024

Dostarczone wymagania :

 Konieczne atrybuty (nazwy kolumn):

origin (lotnisko ICAO) destination (lotnisko ICAO) departure time (rok, miesiac, dzien, godzina) arrival time (rok, miesiac, dzien, godzina) strefa czasowa linia lotnicza model samolotu numer lotu przylot czy wylot scraping date Skraper trzeba ustawić na lokalizację Polską (request z lang=pl,pl).

Skraper powinien mieć delay, co kilka sekund kolejny request

Co jakiś czas deduplikacja - codzienny web scraping powoduje skrapowanie tych samych lotów.



Ten kod generuje listę adresów URL, które prowadzą do informacji o lotach na stronie https://www.flightera.net/pl/airport/.

Lista origin zawiera nazwy portów lotniczych w Polsce wraz z ich kodami IATA. Lista direction zawiera dwa kierunki: "departure" (wylot) i "arrival" (przylot). Zmienna date zawiera datę i czas w formacie 'YYYY-MM-DD HH_MM', które zostaną użyte w wygenerowanych linkach. Funkcja itertools.product() tworzy wszystkie możliwe kombinacje portów lotniczych z listy origin i kierunków z listy direction. Następnie dla każdej kombinacji portu i kierunku generowany jest adres URL, w którym podstawiane są odpowiednie wartości. Ostatecznie wygenerowane adresy URL są wyświetlane za pomocą pętli for.

Kod realizuje następujące funkcje :

Pobieranie danych lotów z różnych lotnisk: Kod ten służy do pobierania danych lotów z różnych lotnisk na podstawie podanych linków do stron internetowych zawierających informacje o lotach.

Przygotowanie danych: Kod parsuje zawartość każdej strony internetowej za pomocą biblioteki BeautifulSoup, aby wyciągnąć informacje o lotach. Następnie oczyszcza te dane i przekształca je w formę, która będzie łatwa do analizy.

Iteracja przez listę linków: Kod przechodzi przez listę linków do stron internetowych z informacjami o lotach. Dla każdego linku:

Wysyła zapytanie HTTP do strony internetowej. Sprawdza, czy odpowiedź jest poprawna (status 200). Analizuje zawartość strony, wyciąga informacje o lotach i oczyszcza je. Tworzy ramkę danych Pandas zawierającą te informacje. Mapuje kolumny na bardziej czytelne nazwy. Dodaje dodatkowe kolumny, takie jak data pobrania danych, kod lotniska początkowego, link i indeks. Zapis do pliku Excel: Po przetworzeniu wszystkich linków, kod łączy wszystkie ramki danych w jedną i zapisuje je do pliku Excel o nazwie 'output.xlsx'.

Opóźnienie między żądaniami: W celu uniknięcia nadmiernego obciążenia serwera, kod zawiera opóźnienie 5 sekund między każdym żądaniem HTTP.

Komunikacja z użytkownikiem: Kod wyświetla również na konsoli informacje o statusie żądań HTTP oraz linki przetwarzane w bieżącym momencie.

Dzięki temu kodowi możemy efektywnie pobrać, przetworzyć i zapisać informacje o lotach z różnych lotnisk do wygodnego formatu pliku Excel.

PROGRAM START

In [1]:
import itertools

origin = ['Szczecin/EPSC', 'Warsaw/EPWA', 'Krakow/EPKK', 'Gdansk/EPGD', 'Katowice/EPKT', 'Wroclaw/EPWR', 'Warsaw/EPMO',
          'Poznan/EPPO','Rzeszow/EPRZ','Bydgoszcz/EPBY','Lublin/EPLB', 'Lodz/EPLL', 'Olsztyn/EPSY','ZielonaGora/EPZG','Warsaw/EPRA']


direction = ['departure', 'arrival']
date = '2023-09-28%2022_25'

# Tworzenie kombinacji portów i kierunków
combinations = list(itertools.product(origin, direction))

# Tworzenie linków dla każdej kombinacji
links = [f"https://www.flightera.net/pl/airport/{port}/{dir}/{date}" for port, dir in combinations]

# Zapisanie uzyskanych linków do listy pythona
links_list = links

# Wyświetlenie wyników tzn zawartości listy linków
for link in links:
    print(link)


# Wyświetlenie listy lotnisk uzytych do utworzenia listy linków
for miasto  in origin:
    print(miasto)




# to do : (tematy do dyskusji)

# wprowadzić mechanizm deduplikacji danych (jaki ?)
# dodac dane z typie samolotu, dostępne z osobnej strony z osobnym urlem do pozyskania  : np.
# https://www.flightera.net/pl/flight_details/LOT+-+Polish+Airlines/LO3932/EPSC/2023-09-29
# i trzeba uzyskac dostęp do tekstu z modelem samolotu np.
#
#MODEL
#SP-LID
#Embraer ERJ175


import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time  #dodano moduł time do obsługi opóźnienia


# z uwagi na polaczenie scrapowania tablicy odlotów i przylotów w jednym scraperze (czy nie powinno byc osobnych scraperów dla odlotów i przylotów ?) przyjęto nazewnictwo :

# origin to nazwa lotniska z którego scrapujemy dane z tablicy odlotów i tablicy przylotów

# to_or_from to nazwa lotniska przeciwstawnego tzn dla odlotów jest "to" a dla przylotów jest to "from" (nazywane również destination)


origin = ['Szczecin/EPSC', 'Warsaw/EPWA', 'Krakow/EPKK', 'Gdansk/EPGD', 'Katowice/EPKT', 'Wroclaw/EPWR', 'Warsaw/EPMO',
          'Poznan/EPPO','Rzeszow/EPRZ','Bydgoszcz/EPBY','Lublin/EPLB', 'Lodz/EPLL', 'Olsztyn/EPSY','ZielonaGora/EPZG','Warsaw/EPRA']

# lista poniżej
#potrzebna do uzyskania nazwy lotniska z którego scrapujemy dane

doubled_origin = [
    'Szczecin/EPSC', 'Szczecin/EPSC', 'Warsaw/EPWA', 'Warsaw/EPWA', 'Krakow/EPKK', 'Krakow/EPKK', 'Gdansk/EPGD',
    'Gdansk/EPGD', 'Katowice/EPKT', 'Katowice/EPKT', 'Wroclaw/EPWR', 'Wroclaw/EPWR', 'Warsaw/EPMO', 'Warsaw/EPMO',
    'Poznan/EPPO', 'Poznan/EPPO', 'Rzeszow/EPRZ', 'Rzeszow/EPRZ', 'Bydgoszcz/EPBY', 'Bydgoszcz/EPBY',
    'Lublin/EPLB', 'Lublin/EPLB', 'Lodz/EPLL', 'Lodz/EPLL', 'Olsztyn/EPSY', 'Olsztyn/EPSY', 'ZielonaGora/EPZG',
    'ZielonaGora/EPZG', 'Warsaw/EPRA', 'Warsaw/EPRA'
]


# lista uzyskana linków uzyskana za pomocą intertools

"""

links_list = ['https://www.flightera.net/pl/airport/Szczecin/EPSC/departure/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Szczecin/EPSC/arrival/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Warsaw/EPWA/departure/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Warsaw/EPWA/arrival/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Krakow/EPKK/departure/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Krakow/EPKK/arrival/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Gdansk/EPGD/departure/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Gdansk/EPGD/arrival/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Katowice/EPKT/departure/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Katowice/EPKT/arrival/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Wroclaw/EPWR/departure/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Wroclaw/EPWR/arrival/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Warsaw/EPMO/departure/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Warsaw/EPMO/arrival/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Poznan/EPPO/departure/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Poznan/EPPO/arrival/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Rzeszow/EPRZ/departure/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Rzeszow/EPRZ/arrival/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Bydgoszcz/EPBY/departure/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Bydgoszcz/EPBY/arrival/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Lublin/EPLB/departure/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Lublin/EPLB/arrival/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Lodz/EPLL/departure/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Lodz/EPLL/arrival/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Olsztyn/EPSY/departure/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Olsztyn/EPSY/arrival/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/ZielonaGora/EPZG/departure/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/ZielonaGora/EPZG/arrival/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Warsaw/EPRA/departure/2023-09-28%2022_25',
 'https://www.flightera.net/pl/airport/Warsaw/EPRA/arrival/2023-09-28%2022_25']


 """

headers = {
    'Accept-Language': "pl,en;q=0.9,en-GB;q=0.8,en-US;q=0.7",
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.43"
}

magazyn = []

for index, link in enumerate(links_list):

#for link in links_list:
    response = requests.get(link, headers=headers)
    print('Kod odpowiedzi:', response.status_code)

    print(link)

    if response.status_code != 200:
        print(f"Failed to fetch data from: {link}")
        continue

    website = BeautifulSoup(response.text, 'html.parser')
    tr = website.find_all("tr")

    cleaned_strings = []

    for t in tr:
        s = (t.text.strip().replace("\n", ";"))
        s = ' '.join(s.split())
        s = s.replace("; ;", ";").replace(";;", ";").replace(";;", ";").replace(";;", ";")
        cleaned_strings.append(s)

    df = pd.DataFrame({'cleaned_text': cleaned_strings})

    split_df = df['cleaned_text'].str.split(';', expand=True)
    split_df.columns = [f"Column_{i+1}" for i in range(split_df.shape[1])]
    result_df = pd.concat([df, split_df], axis=1)

    df_without_first_row = result_df.iloc[1:]
    df_without_empty_rows = df_without_first_row.dropna()
    df_without_empty_rows.index.name = 'Row'

    column_mapping = {
        'cleaned_text': 'cleaned_text',
        'Column_1': 'date',
        'Column_2': 'Column_2',
        'Column_3': 'time',
        'Column_4': 'time_zone',
        'Column_5': 'status',
        'Column_6': 'flight_symbol1',
        'Column_7': 'flight_symbol2',
        'Column_8': 'airlines_name',
        'Column_9': 'airlines_symbol',
        'Column_10': 'to_or_from',
        'Column_11': 'scheduled_departure_CEST',
        'Column_12': 'scheduled_departure_CEST_zone',
        'Column_13': 'scheduled_departure_UTC',
        'Column_14': 'departured',
        'Column_15': 'departured_zone',
        'Column_16': 'departured_time_result',
        'Column_17': 'arrived',
        'Column_18': 'arrived_zone',
        'Column_19': 'arrived_time_result',
        'Column_20': 'duration'
    }
    df_new = df_without_empty_rows.rename(columns=column_mapping)

    df_new['scraping_date'] = datetime.datetime.now()

    df_new['origin'] = doubled_origin[index]

    df_new['link'] = link

    df_new['index'] = index

    #wprowadzono kolumny z kodami lotnisk : origin (lotnisko ICAO) destination (lotnisko ICAO)

    df_new['origin_airport_ICAO'] = df_new['origin'].str.split('/').str[-1].str[-4:] #symbol czteroliterowy ICAO wyciety z kolumny  "origin"


    df_new['to_or_from_airport_ICAO'] =  df_new['to_or_from'].str.extract(r'/\s*([A-Za-z0-9]{4})\)$') #symbol czteroliterowy ICAO wyciety z kolumny "to_or_from"


    #wprowadzono kolumnę z informacją : przylot czy wylot

    #użyto metody apply z wyrażeniem lambda do określenia wartości w nowej kolumnie "departure_arrival"

    #df_new['departure_arrival'] = df_new['index'].apply(lambda x: 'departure' if x % 2 == 0 else None)

    df_new['departure_arrival'] = df_new['index'].apply(lambda x: 'odlot' if x % 2 == 0 else 'przylot')



    # pusta kolumna do usuniecia po połaczeniu ramek

    df_new['Column_2'] = "blank"

    # koniec dodawanych kolumn


    magazyn.append(df_new)

    time.sleep(5)  # Opóźnienie 5 sekund pomiędzy żądaniami


#połączenie wszystkich ramek danych znajdujących się w magazynie.


if magazyn:
    result_df = pd.concat(magazyn, ignore_index=True)
else:
    result_df = pd.DataFrame()
    print ("empty!")


# usuwanie pustej kolumny

result_df=result_df.drop(columns=['Column_2'])


# zapisanie wyniku do excela
result_df.to_excel('output_stan_na_20_03_2024.xlsx', index=False)

print("Zapisano do excela!")

print(result_df)

result_df


https://www.flightera.net/pl/airport/Szczecin/EPSC/departure/2023-09-28%2022_25
https://www.flightera.net/pl/airport/Szczecin/EPSC/arrival/2023-09-28%2022_25
https://www.flightera.net/pl/airport/Warsaw/EPWA/departure/2023-09-28%2022_25
https://www.flightera.net/pl/airport/Warsaw/EPWA/arrival/2023-09-28%2022_25
https://www.flightera.net/pl/airport/Krakow/EPKK/departure/2023-09-28%2022_25
https://www.flightera.net/pl/airport/Krakow/EPKK/arrival/2023-09-28%2022_25
https://www.flightera.net/pl/airport/Gdansk/EPGD/departure/2023-09-28%2022_25
https://www.flightera.net/pl/airport/Gdansk/EPGD/arrival/2023-09-28%2022_25
https://www.flightera.net/pl/airport/Katowice/EPKT/departure/2023-09-28%2022_25
https://www.flightera.net/pl/airport/Katowice/EPKT/arrival/2023-09-28%2022_25
https://www.flightera.net/pl/airport/Wroclaw/EPWR/departure/2023-09-28%2022_25
https://www.flightera.net/pl/airport/Wroclaw/EPWR/arrival/2023-09-28%2022_25
https://www.flightera.net/pl/airport/Warsaw/EPMO/departure/2023-09

,cleaned_text,date,time,time_zone,status,flight_symbol1,flight_symbol2,airlines_name,airlines_symbol,to_or_from,...,arrived_zone,arrived_time_result,duration,scraping_date,origin,link,index,origin_airport_ICAO,to_or_from_airport_ICAO,departure_arrival
0,"Pią, 29. Wrz 2023; ;29 Wrz 08:50;CEST; Wylądow...","Pią, 29. Wrz 2023",29 Wrz 08:50,CEST,Wylądował,LO3932,LOT3932,LOT - Polish Airlines,LO/LOT,Warsaw (WAW / EPWA),...,CEST,19 min przed czasem,43m,2024-03-20 12:38:40.482599,Szczecin/EPSC,https://www.flightera.net/pl/airport/Szczecin/...,0,EPSC,EPWA,odlot
1,"Pią, 29. Wrz 2023; ;29 Wrz 10:00;CEST; Wylądow...","Pią, 29. Wrz 2023",29 Wrz 10:00,CEST,Wylądował,FR2467,RYR2VJ,Ryanair,FR/RYR,London (STN / EGSS),...,BST,3 min po czasie,1h 42m,2024-03-20 12:38:40.482599,Szczecin/EPSC,https://www.flightera.net/pl/airport/Szczecin/...,0,EPSC,EGSS,odlot
2,"Pią, 29. Wrz 2023; ;29 Wrz 10:25;CEST; Wylądow...","Pią, 29. Wrz 2023",29 Wrz 10:25,CEST,Wylądował,W62159,WZZ861G,Wizz Air,W6/WZZ,Oslo (TRF / ENTO),...,CEST,23 min przed czasem,1h 3m,2024-03-20 12:38:40.482599,Szczecin/EPSC,https://www.flightera.net/pl/airport/Szczecin/...,0,EPSC,ENTO,odlot
3,"Pią, 29. Wrz 2023; ;29 Wrz 13:50;CEST; Wylądow...","Pią, 29. Wrz 2023",29 Wrz 13:50,CEST,Wylądował,LO3934,LOT3HA,LOT - Polish Airlines,LO/LOT,Warsaw (WAW / EPWA),...,CEST,11 min przed czasem,41m,2024-03-20 12:38:40.482599,Szczecin/EPSC,https://www.flightera.net/pl/airport/Szczecin/...,0,EPSC,EPWA,odlot
4,"Pią, 29. Wrz 2023; ;29 Wrz 21:10;CEST; Wylądow...","Pią, 29. Wrz 2023",29 Wrz 21:10,CEST,Wylądował,LO3936,LOT9KM,LOT - Polish Airlines,LO/LOT,Warsaw (WAW / EPWA),...,CEST,20 min przed czasem,42m,2024-03-20 12:38:40.482599,Szczecin/EPSC,https://www.flightera.net/pl/airport/Szczecin/...,0,EPSC,EPWA,odlot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473,"Pią, 29. Wrz 2023; ;29 Wrz 11:25;CEST; Wylądow...","Pią, 29. Wrz 2023",29 Wrz 11:25,CEST,Wylądował,LO3529,LOT2XB,LOT - Polish Airlines,LO/LOT,Gdansk (GDN / EPGD),...,CEST,14 min przed czasem,36m,2024-03-20 12:41:07.998826,ZielonaGora/EPZG,https://www.flightera.net/pl/airport/ZielonaGo...,27,EPZG,EPGD,przylot
474,"Pią, 29. Wrz 2023; ;29 Wrz 21:05;CEST; Wylądow...","Pią, 29. Wrz 2023",29 Wrz 21:05,CEST,Wylądował,LO3985,LOT3LX,LOT - Polish Airlines,LO/LOT,Warsaw (WAW / EPWA),...,CEST,14 min przed czasem,38m,2024-03-20 12:41:07.998826,ZielonaGora/EPZG,https://www.flightera.net/pl/airport/ZielonaGo...,27,EPZG,EPWA,przylot
475,"Pią, 29. Wrz 2023; ;29 Wrz 13:30;CEST; Wylądow...","Pią, 29. Wrz 2023",29 Wrz 13:30,CEST,Wylądował,LO2205,LOT2205,LOT - Polish Airlines,LO/LOT,Rome (FCO / LIRF),...,CEST,11 min przed czasem,1h 40m,2024-03-20 12:41:13.423671,Warsaw/EPRA,https://www.flightera.net/pl/airport/Warsaw/EP...,28,EPRA,LIRF,odlot
476,"Pią, 29. Wrz 2023; ;29 Wrz 19:50;CEST; Wylądow...","Pią, 29. Wrz 2023",29 Wrz 19:50,CEST,Wylądował,LO2201,LOT2201,LOT - Polish Airlines,LO/LOT,Paris (CDG / LFPG),...,CEST,23 min przed czasem,1h 55m,2024-03-20 12:41:13.423671,Warsaw/EPRA,https://www.flightera.net/pl/airport/Warsaw/EP...,28,EPRA,LFPG,odlot



Oto szczegółowy opis funkcjonalności tego kodu:

Pobieranie danych lotów z różnych lotnisk: Kod ten służy do pobierania danych lotów z różnych lotnisk na podstawie podanych linków do stron internetowych zawierających informacje o lotach.

Przygotowanie danych: Kod parsuje zawartość każdej strony internetowej za pomocą biblioteki BeautifulSoup, aby wyciągnąć informacje o lotach. Następnie oczyszcza te dane i przekształca je w formę, która będzie łatwa do analizy.

Iteracja przez listę linków: Kod przechodzi przez listę linków do stron internetowych z informacjami o lotach. Dla każdego linku:

Wysyła zapytanie HTTP do strony internetowej.
Sprawdza, czy odpowiedź jest poprawna (status 200).
Analizuje zawartość strony, wyciąga informacje o lotach i oczyszcza je.
Tworzy ramkę danych Pandas zawierającą te informacje.
Mapuje kolumny na bardziej czytelne nazwy.
Dodaje dodatkowe kolumny, takie jak data pobrania danych, kod lotniska początkowego, link i indeks.
Zapis do pliku Excel: Po przetworzeniu wszystkich linków, kod łączy wszystkie ramki danych w jedną i zapisuje je do pliku Excel o nazwie 'output.xlsx'.

Opóźnienie między żądaniami: W celu uniknięcia nadmiernego obciążenia serwera, kod zawiera opóźnienie 5 sekund między każdym żądaniem HTTP.

Komunikacja z użytkownikiem: Kod wyświetla również na konsoli informacje o statusie żądań HTTP oraz linki przetwarzane w bieżącym momencie.

Dzięki temu kodowi możemy efektywnie pobrać, przetworzyć i zapisać informacje o lotach z różnych lotnisk do wygodnego formatu pliku Excel. Jest to przydatne szczególnie dla osób śledzących loty lub zajmujących się analizą danych lotniczych.